# [실습 1] Built-in & Third-party Tool 사용하기

학습 내용:
1. 환경 설정: 필요한 라이브러리 설치 및 API 키 설정
2. Third-party Tool: Tavily 웹 검색 도구 사용법
3. Built-in Tool: LangChain에서 기본 제공하는 DuckDuckGo 검색 도구 사용법
4. Toolkit: 여러 도구를 한번에 가져오는 Toolkit 활용법 (Wikipedia)
5. Agent 생성: 정의된 도구들을 활용하여 스스로 추론하고 답을 찾는 Agent 만들기

## 1. 환경 설정

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [ ]:
# LangChain 관련 기본 import
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor

# LLM 모델을 초기화합니다. 이번 실습에서는 'gpt-4o-mini'를 사용합니다.
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 2. Third-party Tool: Tavily 웹 검색
- Tavily는 AI Agent를 위해 특별히 설계된 검색 엔진입니다.  
- 일반 검색 엔진보다 더 정확하고 간결한 정보를 제공하여 Agent의 성능을 높여줍니다.

**Tavily API Key 발급 방법**
1. Tavily 공식 홈페이지 접속: 우측 상단의 "Sign in" 또는 "Sign up"을 클릭하여 회원가입을 진행합니다. Google 계정으로 쉽게 가입할 수 있습니다.
공식 사이트 : https://auth.tavily.com/u/login/identifier?state=hKFo2SBETnBjX3pFNy1ubEpzQ0Jtemo5dnhLSm5DTjVsd2pGSKFur3VuaXZlcnNhbC1sb2dpbqN0aWTZIGh5ZWVYbm5TLW5DS2pwbEF4aDJITGhvSElFMTRmQTg3o2NpZNkgUlJJQXZ2WE5GeHBmVFdJb3pYMW1YcUxueVVtWVNUclE

2. API Key 확인: 회원가입 후 대시보드(Dashboard)로 이동하면 "API Key" 섹션이 있습니다. 여기에 표시된 API 키를 복사합니다.

3. 무료 사용: Tavily는 "Researcher" 플랜으로 월 1,000건의 무료 API 호출을 제공하므로, 실습용으로는 충분합니다.

In [ ]:
# API 키 설정
os.environ["TAVILY_API_KEY"] = "YOUR_API_KEY"

In [ ]:
from langchain_tavily import TavilySearch

# Tavily 검색 도구를 생성합니다.
# max_results는 검색 결과의 최대 개수를 지정합니다.
tavily_tool = TavilySearch(
    max_results=5
)

In [ ]:
# Tavily 도구를 직접 호출하여 테스트해볼 수 있습니다.
print("--- Tavily 검색 테스트 ---")
search_result = tavily_tool.invoke({"query": "LangChain이란 무엇인가요?"})
print(search_result)
print("-" * 20)

# 3. Built-in Tool: DuckDuckGo 웹 검색
- LangChain은 다양한 내장 도구를 제공합니다. DuckDuckGo는 그 중 하나로,
- 별도의 API 키 없이 사용할 수 있는 검색 도구입니다.

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

# DuckDuckGo 검색 도구를 생성합니다.
duckduckgo_tool = DuckDuckGoSearchRun()

In [ ]:
# DuckDuckGo 도구를 직접 호출하여 테스트해봅니다.
print("--- DuckDuckGo 검색 테스트 ---")
question = "LangChain이란 무엇인가요?"
search_result_ddg = duckduckgo_tool.# [YOUR CODE]
print(search_result_ddg)
print("-" * 20)

# 4. Toolkit: File Management Toolkit
- Toolkit은 특정 작업과 관련된 여러 도구의 묶음입니다.
https://python.langchain.com/v0.1/docs/integrations/toolkits/

In [ ]:
from langchain_community.agent_toolkits import FileManagementToolkit

# 파일 관리 툴킷 초기화
fm_toolkit = FileManagementToolkit(root_dir="./my_agent_data")

In [ ]:
# Toolkit에서 제공하는 도구들을 가져옵니다.
# get_tools() 메서드는 Toolkit에 포함된 모든 도구의 리스트를 반환합니다.
fm_tools = fm_toolkit.# [YOUR CODE]

print(f"--- File Management Toolkit 도구 목록 ({len(fm_tools)}개) ---")
for tool in fm_tools:
    print(f"도구 이름: {tool.name}, 설명: {tool.description}")
print("-" * 20)

# 5. Agent 생성 및 실행

이제 위에서 정의한 도구들을 Agent에게 부여하여 실제 문제를 해결하게 해보겠습니다.

In [ ]:
# Agent가 사용할 도구 목록을 정의합니다.
# 각 Tool 객체는 이름(name), 실행할 함수(func), 그리고 설명(description)으로 구성됩니다.
# 이 'description'은 Agent가 어떤 상황에 이 도구를 사용해야 할지 판단하는 매우 중요한 기준이 됩니다.

tools = [
    Tool(
        name="Tavily-Search",
        func= # [YOUR CODE]
        description="최신 정보나 시사, 인물, 사실 관계 확인 등 웹 검색이 필요할 때 사용합니다. 다른 도구로 해결할 수 없는 질문에 대한 최종적인 답변 수단으로 유용합니다.",
    ),
    Tool(
        name="DuckDuckGo-Search",
        func= # [YOUR CODE]
        description="Tavily 검색 결과가 만족스럽지 않거나, 다른 관점의 웹 검색이 필요할 때 사용합니다.",
    ),
    # Wikipedia Toolkit의 도구들을 리스트에 추가합니다.
    *fm_tools
]

In [ ]:
# Agent를 만들기 위한 프롬프트를 가져옵니다.
# hub.pull은 LangChain Hub에서 미리 만들어진 프롬프트 객체를 다운로드하는 기능입니다.
# "hwchase17/openai-functions-agent"는 OpenAI의 함수 호출 기능에 최적화된 표준 프롬프트입니다.
prompt = hub.pull("hwchase17/openai-functions-agent")

In [ ]:
# 프롬프트 내용을 확인해볼 수 있습니다.
print(prompt.messages)

In [ ]:
# OpenAI Functions Agent를 생성합니다.
# 이 Agent는 LLM의 함수 호출(Function Calling) 기능을 사용하여 어떤 도구를 언제 호출할지 결정합니다.
agent = create_openai_functions_agent(llm, tools, prompt)

In [ ]:
# AgentExecutor는 Agent의 실행을 담당합니다.
# Agent가 도구를 호출하고, 그 결과를 다시 Agent에게 전달하는 등의 순환적인 과정을 관리합니다.
# verbose=True로 설정하면 Agent의 생각의 흐름(Chain of Thought)을 자세히 볼 수 있습니다.
agent_executor = AgentExecutor(agent= # [YOUR CODE]
                               tools= # [YOUR CODE]
                               verbose=True)

In [ ]:
# 이제 Agent에게 질문을 던져보겠습니다.
# Agent는 질문을 이해하고, 가장 적절한 도구를 선택하여 실행하고, 최종 답변을 생성합니다.
print("\n--- Agent 실행 예시 1 ---")
question1 = "Attention Is All You Need 논문의 저자는 누구인가요? 그들의 소속도 알려주세요."
response1 = agent_executor.# [YOUR CODE]
print("\n[최종 답변]:", response1["output"])

In [ ]:
print("\n--- Agent 실행 예시 2 ---")
question2 = "OpenAI의 gpt-4o-mini 모델과 구글의 Gemini 1.5 Flash 모델을 비교 설명해줘."
response2 = agent_executor.# [YOUR CODE]
print("\n[최종 답변]:", response2["output"])

In [ ]:
print("\n--- Agent 실행 예시 3 ---")
# Wikipedia 도구와 웹 검색 도구를 함께 사용해야 하는 복합적인 질문
question3 = "파일 'answer.txt'를 만들고, 그 안에 대한민국의 수도가 어디인지 검색해서 답을 써줘."
response3 = agent_executor.# [YOUR CODE]
print("\n[최종 답변]:", response3["output"])